Statistical Comparison between Models using Cross-Validation
===

Author: Nathan A. Mahynski

Date: 2023/09/07

Description: Using cross validation to interrogate differences in model performance.

In the previous notebook models were compared using nested cross-validation so that an uncertainty on the generalization error could be obtained.  **The question becomes, how do we know if these differences are statistically significant?** In principle, we can then perform standard hypothesis tests to check since we have means and uncertainties.  However, the main problem with this is that performance estimates are not independent of each other.  This tends to result in higher type I error (incorrectly reject the null hypothesis, which is two algorithms are the same) which means **you tend to see a difference in the performances when there isn't any.**

<!-- Another issue is that many of these approaches do not account for hyperparameter selection.  They tend to use disjoint training sets to train the model and evaluate on a separate test set, but if you have to do hyperameter selection usually the final model is selected based on which performs the best on these test sets.  As a result that test performance is biased - this is the reason a separate validation set is usually introduced.  The methods below neglect this, but [recent work](https://arxiv.org/pdf/1809.09446.pdf) suggests that, while this bias exists, it does not strongly affect the relative ranking of models or pipelines.-->

In [1]:
using_colab = 'google.colab' in str(get_ipython())
if using_colab:
    !pip install git+https://github.com/mahynski/pychemauth@main

try:
    import pychemauth
except:
    raise ImportError("pychemauth not installed")
    
import matplotlib.pyplot as plt
%matplotlib inline

import watermark
%load_ext watermark

%load_ext autoreload
%autoreload 2

In [23]:
import sklearn
import scipy

import numpy as np

from imblearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, RepeatedKFold, train_test_split, RepeatedStratifiedKFold, GridSearchCV
from sklearn import datasets

from pychemauth.analysis.compare import BiasedNestedCV
from pychemauth.analysis.compare import Compare
from pychemauth.preprocessing.scaling import CorrectedScaler

In [3]:
%watermark -t -m -v --iversions

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.14.0

Compiler    : GCC 12.2.0
OS          : Linux
Release     : 6.2.0-26-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 40
Architecture: 64bit

scipy     : 1.11.1
pychemauth: 0.0.0b3
watermark : 2.4.3
numpy     : 1.24.4
matplotlib: 3.7.2
sklearn   : 1.3.0



<h3>Load the Data</h3>

In [4]:
data = datasets.load_wine()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    shuffle=True,
    random_state=0,
    test_size=0.2
)

In [5]:
# Let's re-use these models from the last example.
def make_svc(k_inner, random_state_inner=42):
    pipe_svc = Pipeline(steps=[
        ("scaler", CorrectedScaler()),
        ("model", SVC())
    ])
    gs_svc = GridSearchCV(
        estimator=pipe_svc,
        param_grid=[{'model__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}],
        scoring='accuracy',
        cv=sklearn.model_selection.StratifiedKFold( # Since this is a classification problem we will stratify
            n_splits=k_inner,
            shuffle=True,
            random_state=random_state_inner
        ),
        refit=True
    )
    
    return gs_svc

def make_tree(k_inner, random_state_inner=42):
    gs_tree = GridSearchCV(
        estimator=DecisionTreeClassifier(random_state=0),
        param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, None]}],
        scoring='accuracy',
        cv=sklearn.model_selection.StratifiedKFold( # Since this is a classification problem we will stratify
            n_splits=k_inner,
            shuffle=True,
            random_state=random_state_inner
        ),
        refit=True
    )
    
    return gs_tree

The resampled paired t-test
---

The first way to obtain uncertainties on generalization performance is simply to shuffle the data and repeat the holdout procedure to get many different test set performances.  Statistical significance can be assessed by obtaining the mean and standard deviation for each model and performing a paired [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test) on the differences.  If we use the same data split then both models are tested on the same "chunk" so we should use the paired version. However, a key assumption of the t-test is that measurements are independent; as you can see below for a small subset of data, reusing the data like this means points reoccur in many training (and test) sets, meaning each result is not independent.  This has been [shown](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.37.3325&rep=rep1&type=pdf) to yield an unacceptably high [type I error](https://en.wikipedia.org/wiki/Type_I_and_type_II_errors) (detection of a difference between algorithms when there is none).

> Type I error is the rejection of the null hypothesis.  In [frequentist statistics](https://www.statisticshowto.com/frequentist-statistics/) the null hypothesis is usually formulated so that the onus is one proving a difference between to things (e.g., the difference of 2 means), and is usually [stated as](https://www.statisticshowto.com/probability-and-statistics/null-hypothesis/#state) the currently accepted [truth](https://www.statisticshowto.com/probability-and-statistics/hypothesis-testing/).  The burden of proof then falls on the researcher to disprove this and reject the current status quo in favor of something else.

In [6]:
for loop in range(3):
    X_, y_ = sklearn.utils.shuffle(X_train[:5,:2], y_train[:5], random_state=loop)
    X_train_ = X_[:3]
    X_test_ = X_[3:]
    print('Training set on fold {} :\n {}'.format(loop+1, X_train_))
    print('Test set on fold {} :\n {}'.format(loop+1, X_test_))
    print('---------------------------------')

Training set on fold 1 :
 [[11.62  1.99]
 [13.69  3.26]
 [12.69  1.53]]
Test set on fold 1 :
 [[13.4   3.91]
 [13.5   1.81]]
---------------------------------
Training set on fold 2 :
 [[11.62  1.99]
 [12.69  1.53]
 [13.5   1.81]]
Test set on fold 2 :
 [[13.69  3.26]
 [13.4   3.91]]
---------------------------------
Training set on fold 3 :
 [[11.62  1.99]
 [13.5   1.81]
 [12.69  1.53]]
Test set on fold 3 :
 [[13.4   3.91]
 [13.69  3.26]]
---------------------------------


The k-fold CV paired t-test
---

A slight improvement: split data into k disjoint folds (as in CV), and **using the same data split** evaluate each model.  The problem is now that each data point contributes to exactly $k-1$ training sets; therefore the scores estimated from the remaining fold will still be correlated, in general, because they are trained (and tested) on some of the same data.  However, note that when $k=2$ the data overlap problem goes away! The tradeoff is that we have only 2 estimates of the performance (2 test sets).

In [7]:
cv = sklearn.model_selection.StratifiedKFold(n_splits=2, random_state=42, shuffle=True)

for train_index, test_index in cv.split(X_train[:20], y_train[:20]):
    print(train_index, test_index)

[ 1  6  9 10 11 13 14 16 17 19] [ 0  2  3  4  5  7  8 12 15 18]
[ 0  2  3  4  5  7  8 12 15 18] [ 1  6  9 10 11 13 14 16 17 19]


In [8]:
# We can then evalute a model on each test set using cross-validation
scores = cross_val_score(
    estimator=DecisionTreeClassifier(random_state=0, max_depth=3), 
    X=X_train[:20],
    y=y_train[:20],
    cv=cv
)

This produces k estimates which we can get a mean and uncertainty from.

In [9]:
print('Raw scores: {}\nMean : {}\nSt. Dev. : {}'.format(scores, '%.3f'%np.mean(scores), '%.3f'%np.std(scores)))

Raw scores: [0.6 0.5]
Mean : 0.550
St. Dev. : 0.050


The repeated k-fold CV paired t-test
---

Using $k=2$ helped remove the some bias due to re-use of data in training sets, but 2 error estimates is really not enough to be very helpful.  [Dietterich popularized](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.37.3325&rep=rep1&type=pdf) a 5x2 repeated CV scheme.  This essentially combines both ideas from the last section by shuffling the data differently 5 times, then doing 2-fold CV in each shuffle.  Of course, this re-uses data due to the reshuffling and so estimates are not completely independent, but was shown to have a tolerably elevated [Type I error rate](https://en.wikipedia.org/wiki/Type_I_and_type_II_errors).  Still, this gives us 10 estimates instead of 2.

In [10]:
# Here is a simple example of how this splitting occurs
X_dummy = X_train[:10] 
y_dummy = y_train[:10] 

rkf = RepeatedKFold(n_splits=2, n_repeats=5, random_state=0)
rkf.get_n_splits(X_dummy, y_dummy)

for i, (train_index, test_index) in enumerate(rkf.split(X_dummy)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold 0:
  Train: index=[0 3 5 6 7]
  Test:  index=[1 2 4 8 9]
Fold 1:
  Train: index=[1 2 4 8 9]
  Test:  index=[0 3 5 6 7]
Fold 2:
  Train: index=[0 4 6 7 8]
  Test:  index=[1 2 3 5 9]
Fold 3:
  Train: index=[1 2 3 5 9]
  Test:  index=[0 4 6 7 8]
Fold 4:
  Train: index=[0 1 6 7 9]
  Test:  index=[2 3 4 5 8]
Fold 5:
  Train: index=[2 3 4 5 8]
  Test:  index=[0 1 6 7 9]
Fold 6:
  Train: index=[0 3 4 5 8]
  Test:  index=[1 2 6 7 9]
Fold 7:
  Train: index=[1 2 6 7 9]
  Test:  index=[0 3 4 5 8]
Fold 8:
  Train: index=[0 3 6 8 9]
  Test:  index=[1 2 4 5 7]
Fold 9:
  Train: index=[1 2 4 5 7]
  Test:  index=[0 3 6 8 9]


In [11]:
# Here is how 5x2 repeated CV is performed
rkf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=0)

pipe1 = pipe_svc = Pipeline(steps=[
            ("scaler", CorrectedScaler()),
            ("model", SVC(C=1.0))
        ])
pipe2 = DecisionTreeClassifier(random_state=0, max_depth=3)

scores1 = []
scores2 = []
for train_index, test_index in rkf.split(X_train, y_train):
    X_train_, X_test_ = X_train[train_index], X_train[test_index]
    y_train_, y_test_ = y_train[train_index], y_train[test_index]

    # Train both pipelines on the same data and evaluate on the same test set
    pipe1.fit(X_train_, y_train_)
    scores1.append(pipe1.score(X_test_, y_test_))

    pipe2.fit(X_train_, y_train_)
    scores2.append(pipe2.score(X_test_, y_test_))

In [12]:
# We can take the difference from each run because both pipelines tested/trained on the same data each time
difference = np.array(scores2) - np.array(scores1)

In [13]:
difference

array([-0.07042254, -0.12676056, -0.11267606, -0.07042254, -0.05633803,
        0.        , -0.05633803, -0.05633803, -0.05633803,  0.        ])

In [63]:
# You can get identical results from cross_val_score like this
scores1 = cross_val_score(
    estimator=pipe1, 
    X=X_train,
    y=y_train,
    cv=rkf,
)

scores2 = cross_val_score(
    estimator=pipe2, 
    X=X_train,
    y=y_train,
    cv=rkf
)

difference = np.array(scores2) - np.array(scores1)

In [64]:
difference

array([-0.07042254, -0.12676056, -0.11267606, -0.07042254, -0.05633803,
        0.        , -0.05633803, -0.05633803, -0.05633803,  0.        ])

In [65]:
# PyChemAuth has a built in function to perform this.
# Be sure any random states are set in your pipelines and to set the random_state here as well.
scores1, scores2 = Compare.repeated_kfold(pipe1, pipe2, X_train, y_train, n_repeats=5, k=2, random_state=0)
difference = np.array(scores2) - np.array(scores1)
difference

array([-0.07042254, -0.12676056, -0.11267606, -0.07042254, -0.05633803,
        0.        , -0.05633803, -0.05633803, -0.05633803,  0.        ])

**Importantly,** the t-statistic used by Dietterich for his 5x2 CV t-test is modified from a conventional t-statistic, but we will not use it here anyway.  A example implementation is discussed [here](https://www.kaggle.com/code/ogrellier/parameter-tuning-5-x-2-fold-cv-statistical-test/notebook).

In general we can perform CV any number of times to get many test performances to improve our assessment of the relative performance. Assuming $n = r \times k$ runs have been performed, we can define the difference in test set performances between algorithms A and B for a fold in a certain repeat as $x_j = A_j - B_j$, and $\hat{s}^2 = \frac{1}{n-1}\sum(x_j - \bar{x})^2$ (unbiased sample variance).  This is used with the Student's t distribution and $n-1$ degrees of freedom.

$$
t = \frac{ \frac{1}{n} \sum_{j=1}^n x_j}{\sqrt{ \hat{s}^2 } /\sqrt{n} } =  \frac{ \frac{1}{n} \sum_{j=1}^n x_j}{\sqrt{ \frac{1}{n} \hat{s}^2 } }
$$

Dietterich originally found that the standard t-statistic was elevated and therefore needed some correction so he used a different formula;  [Nadeau & Bengio suggested a different correction](https://link.springer.com/article/10.1023/A:1024068626366) to the t-statistic which is designed to "overestimate the variance to yield conservative inference." This has a similar effect of deflating the t-statistic to account for correlations between the data folds. Here, $\rho = 1/k$ is the fraction of the data used for training.

$$
t_{\rm corrected} =  \frac{ \frac{1}{n} \sum_{j=1}^n x_j}{\sqrt{ \left( \frac{1}{n} + \frac{\rho}{1 - \rho} \right) \hat{s}^2 } }
$$

In 2004, [Bouckaert & Frank](https://link.springer.com/chapter/10.1007/978-3-540-24775-3_3) argued that reproducility is more important than minimizing type I error; this depends strongly on how the data is partitioned since different researchers will probably have different RNG seeds when trying to repeat each others' work. Instead they recommend either (a) 100 runs of random resampling or (b) 10×10 repeated cross-validation with the Nadeau and Bengio correction (discussed above).  Interestingly, (a) is the worst thing you can do according to Dietterich, so the latter is recommended.

In [66]:
def significance(alpha, scores_svc, scores_tree, n_repeats):
    """Check the significance of any differences."""
    p_value = Compare.corrected_t(
        scores_svc, # One with the best average
        scores_tree,
        n_repeats=n_repeats 
    )

    if p_value < alpha: # Reject H0
        print('SVC outperforms the Tree ({} < {})'.format('%.4f'%p_value, '%.2f'%alpha)) 
    else: # Fail to reject H0
        print('Pipelines are about the same ({} >= {})'.format('%.4f'%p_value, '%.2f'%alpha))

In [67]:
# You can perform a 2-sided test to check if algorithms are different; however, it is often the case that we develop a 
# simpler algorithm that might appear to perform a bit worse than some state-of-the-art one, and we need to check if
# its underperformance is actually statistically significant.  For this we can perform 1-sided test to see if the mean
# performance of the model with the better average is, in fact, better than the one that is worse.

# Here the SVC (scores1) seem to be better than the decision tree (scores2).  Let's do a 1-sided hypothesis test.
scores1, scores2 = Compare.repeated_kfold(pipe1, pipe2, X_train, y_train, n_repeats=5, k=2, random_state=0)

print('%dx%d Nested CV accuracy for SVC: %.3f +/- %.3f' % (5, 2, np.mean(scores1), np.std(scores1)))
print('%dx%d Nested CV accuracy for Tree: %.3f +/- %.3f' % (5, 2, np.mean(scores2), np.std(scores2)))

significance(alpha=0.05, scores_svc=scores1, scores_tree=scores2, n_repeats=5)

5x2 Nested CV accuracy for SVC: 0.968 +/- 0.027
5x2 Nested CV accuracy for Tree: 0.907 +/- 0.031
Pipelines are about the same (0.0934 >= 0.05)


In [73]:
# 10x10 is the recommendation, and clearly yields different results!
scores1, scores2 = Compare.repeated_kfold(pipe1, pipe2, X_train, y_train, n_repeats=10, k=10, random_state=0)

print('%dx%d Nested CV accuracy for SVC: %.3f +/- %.3f' % (5, 2, np.mean(scores1), np.std(scores1)))
print('%dx%d Nested CV accuracy for Tree: %.3f +/- %.3f' % (5, 2, np.mean(scores2), np.std(scores2)))

significance(alpha=0.05, scores_svc=scores1, scores_tree=scores2, n_repeats=10)

5x2 Nested CV accuracy for SVC: 0.978 +/- 0.037
5x2 Nested CV accuracy for Tree: 0.930 +/- 0.059
SVC outperforms the Tree (0.0103 < 0.05)


The (repeated) nested k-fold paired t-test
---

In the previous notebook we used nested CV because we needed to get an error estimate that accounts for **both the model and its hyperparameter search.**  Until now we have just been comparing pipelines with fixed hyperparameters; often we need to optimize those as well, so nested CV is necessary.  In principle, we can simply apply the corrected t-statistic to the results from the test folds generated by nested CV. (Image credit to [Python Machine Learning, 3rd Ed.](https://github.com/rasbt/python-machine-learning-book-3rd-edition/) by Raschka and Mirjalili.)

<!-- ![title](https://github.com/rasbt/python-machine-learning-book-3rd-edition/blob/master/ch06/images/06_07.png?raw=true){ width=500px } -->

<img src="https://github.com/rasbt/python-machine-learning-book-3rd-edition/blob/master/ch06/images/06_07.png?raw=true" style="width:500px;">

In this 5x2 nested CV example, we would have 5 test set estimates (r=1, k=5) and can use the corrected t-test above, and/or repeat this several times so that r > 1.

There is sometimes confusion surrounding the terminology since nested CV is sometimes referred to as $m \times n$ *nested* CV, while repeated CV is just called $m \times n$ CV.  Both are sometimes referred to as simply $5 \times 2$ CV, but they are **not the same.**

See this stackexchange [discussion](https://stats.stackexchange.com/questions/151710/nested-cross-validation-how-is-it-different-from-model-selection-via-kfold-cv).

In [54]:
# Nested CV is equivalent to using the repeated k-fold with only 1 repeat, assuming we pass grid search objects 
# equipped with CV splitters.

def compare(k_inner, k_outer, n_repeats):
    scores1, scores2 = Compare.repeated_kfold(
        pipe1=GridSearchCV(
            estimator=Pipeline(steps=[
                ("scaler", CorrectedScaler()),
                ("model", SVC())
            ]),
            param_grid=[{'model__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}],
            scoring='accuracy',
            cv=sklearn.model_selection.StratifiedKFold( # Since this is a classification problem we will stratify
                n_splits=k_inner, # Inner part of the nested CV is handled by this object
                shuffle=True,
                random_state=0
            ),
            refit=True # We should refit on the entire training set once optimal hyperparameters are found
        ), 
        pipe2=GridSearchCV(
            estimator=DecisionTreeClassifier(random_state=0),
            param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, None]}],
            scoring='accuracy',
            cv=sklearn.model_selection.StratifiedKFold( # Since this is a classification problem we will stratify
                n_splits=k_inner, # Inner part of the nested CV is handled by this object
                shuffle=True,
                random_state=0
            ),
            refit=True # We should refit on the entire training set once optimal hyperparameters are found
        ), 
        X=X_train, 
        y=y_train, 
        n_repeats=n_repeats, 
        k=k_outer, 
        random_state=0
    )

    return scores1, scores2

Let's review in detail what is happening above.

1. `Compare.repeated_kfold` is doing only 1 repeat of a k-fold split of the data it is given.  This is the outer split (k_outer).
2. Each estimator is a `GridSearchCV` object which uses k-fold CV to do the inner loop: (1) find optimal hyperparameters, then (2) refit on the entire training fold it was given.
3. Each model is optimized, then tested on a single outer test fold so there are k_outer scores.

In fact, by specifying `n_repeats` to be more than one, we can simply perform repeated (nested) k-fold CV where we have an internal nested loop to handle hyperparameter optimization.

In [74]:
# This is just nested k-fold CV where k = k_outer = 5
scores1, scores2 = compare(k_inner=2, k_outer=5, n_repeats=1)

print('%dx%d Nested CV accuracy for SVC: %.3f +/- %.3f' % (5, 2, np.mean(scores1), np.std(scores1)))
print('%dx%d Nested CV accuracy for Tree: %.3f +/- %.3f' % (5, 2, np.mean(scores2), np.std(scores2)))

significance(alpha=0.05, scores_svc=scores1, scores_tree=scores2, n_repeats=1)

5x2 Nested CV accuracy for SVC: 0.979 +/- 0.041
5x2 Nested CV accuracy for Tree: 0.915 +/- 0.043
Pipelines are about the same (0.0900 >= 0.05)


In [76]:
# This is repeated nested k-fold CV where k = k_outer = 5
# Similar results were obtained for 10x10 repeated CV
scores1, scores2 = compare(k_inner=2, k_outer=5, n_repeats=20)

print('%dx%d Nested CV accuracy for SVC: %.3f +/- %.3f' % (5, 2, np.mean(scores1), np.std(scores1)))
print('%dx%d Nested CV accuracy for Tree: %.3f +/- %.3f' % (5, 2, np.mean(scores2), np.std(scores2)))

significance(alpha=0.05, scores_svc=scores1, scores_tree=scores2, n_repeats=20)

5x2 Nested CV accuracy for SVC: 0.975 +/- 0.026
5x2 Nested CV accuracy for Tree: 0.924 +/- 0.046
SVC outperforms the Tree (0.0194 < 0.05)


In [77]:
# 10x10 repeated nested CV could look like this; note k_inner should be appropriate for good hyperparameter tuning
# but neither of these "10s" refer to this.
scores1, scores2 = compare(k_inner=5, k_outer=10, n_repeats=10)

print('%dx%d Nested CV accuracy for SVC: %.3f +/- %.3f' % (5, 10, np.mean(scores1), np.std(scores1)))
print('%dx%d Nested CV accuracy for Tree: %.3f +/- %.3f' % (5, 10, np.mean(scores2), np.std(scores2)))

significance(alpha=0.05, scores_svc=scores1, scores_tree=scores2, n_repeats=10)

5x2 Nested CV accuracy for SVC: 0.973 +/- 0.041
5x2 Nested CV accuracy for Tree: 0.928 +/- 0.063
SVC outperforms the Tree (0.0248 < 0.05)


In [ ]:
# It seems that doing more and more testing suggest that the SVC pipeline outperforms the Tree; let's examine this in 
# more detail. 

p = {}
for n_repeats in np.arange(1, 10+1, 2):
    p[n_repeats] = []
    for k_outer in [2, 4, 6, 8, 10, 12, 14, 16]:
        scores1, scores2 = compare(k_inner=5, k_outer=k_outer, n_repeats=n_repeats)
        
        p[n_repeats].append(Compare.corrected_t(
            scores1, 
            scores2,
            n_repeats=n_repeats 
        ))

In [ ]:
# As long as k_outer is ~ 5 or higher, the number of repeats doesn't really affect the p value.

# Only very few repeats with very small k_outer (which lead to a small number of test sets, N_tot = N_r * k_outer)
# result in p > 0.05 (i.e., the belief that these pipelines perform similarly).  The p-value converges as
# N_tot increases.

for n in p.keys():
    plt.plot([2, 4, 6, 8, 10, 12, 14, 16], p[n], 'o-', label='N_r = {}'.format(n))
plt.xlabel('k_outer')
plt.ylabel('P')
plt.legend(loc='best')
_ = plt.axhline(0.05, color='red')

In [ ]:
def compare_biased_ncv(k, k_train=5):
    ncv = BiasedNestedCV(k_inner=k_train, k_outer=k)
    scores_svc = ncv.grid_search(
        pipeline = Pipeline(steps=[
            ("scaler", CorrectedScaler()),
            ("model", SVC())
        ]),
        param_grid=[{'model__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}],
        X=X_train,
        y=y_train,
        classification=True,
    )
    
    ncv = BiasedNestedCV(k_inner=k_train, k_outer=k)
    scores_tree = ncv.grid_search(
        pipeline = Pipeline(steps=[
            ("tree", DecisionTreeClassifier(random_state=0)),
        ]),
        param_grid=[{'tree__max_depth': [1, 2, 3, 4, 5, 6, 7, None]}],
        X=X_train,
        y=y_train,
        classification=True,
    )
    
    return scores_svc, scores_tree

In [ ]:
p_biased = []
for k_outer in [2, 4, 6, 8, 10, 12, 14, 16]:
    scores_svc, scores_tree = compare_biased_ncv(k_outer)
    
    p_biased.append(
        Compare.corrected_t(
            scores1=scores_svc, 
            scores2=scores_tree,
            n_repeats=k_outer
        )
    )
            

In [ ]:
for n in p.keys():
    plt.plot([2, 4, 6, 8, 10], p[n], 'o-', label='N_r = {}'.format(n))
plt.xlabel('k_outer')
plt.ylabel('P')
plt.legend(loc='best')
_ = plt.axhline(0.05, color='red')

plt.plot([2, 4, 6, 8, 10], p_biased, label='Biased NCV')

In [ ]:
# Since we typically do atleast 10 outer folds these all appear similar

Rashomon Sets
---

In [ ]:
Rudin papers

In [ ]:
# visualize

Bayesian Comparison
---

See "Time for a Change: a Tutorial for Comparing Multiple Classifiers Through Bayesian Analysis", Alessio Benavoli, Giorgio Corani, Janez Demšar, Marco Zaffalon. Journal of Machine Learning Research, 18 (2017) 1-36.

They have an associated [github repo](https://github.com/janezd/baycomp) and [documentation](https://baycomp.readthedocs.io/en/latest/) that discusses the advantages of using Bayesian methods to compare ML pipelines instead of frequentist statistics (hypothesis testing).

More discussion [here](https://www.statisticshowto.com/probability-and-statistics/hypothesis-testing/) on the argument for (and against) using Bayesian statistics instead of frequentist.